In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gradio as gr
from datetime import datetime
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

**Find the information about dataset from the below Link** - https://www.kaggle.com/datasets/gsagar12/dspp1

In [3]:
# Load CSVs
cust_cases_df = pd.read_csv('https://raw.githubusercontent.com/Namachivayam2001/Public_Datasets/refs/heads/main/customer_cases.csv')
cust_info_df = pd.read_csv('https://github.com/Namachivayam2001/Public_Datasets/raw/main/customer_info.csv')
cust_prod_df = pd.read_csv(r'D:\python\Datasets\customer_product.csv')
prod_info_df = pd.read_csv('https://github.com/Namachivayam2001/Public_Datasets/raw/main/product_info.csv')

In [4]:
# check the column names of the tables
print(f'cust_cases: {cust_cases_df.columns}')
print(f'cust_info: {cust_info_df.columns}')
print(f'cust_prod: {cust_prod_df.columns}')
print(f'prod_info: {prod_info_df.columns}')

cust_cases: Index(['Unnamed: 0', 'case_id', 'date_time', 'customer_id', 'channel',
       'reason'],
      dtype='object')
cust_info: Index(['Unnamed: 0', 'customer_id', 'age', 'gender'], dtype='object')
cust_prod: Index(['Unnamed: 0', 'customer_id', 'product', 'signup_date_time',
       'cancel_date_time'],
      dtype='object')
prod_info: Index(['product_id', 'name', 'price', 'billing_cycle'], dtype='object')


In [5]:
cust_cases_df.drop(columns=['Unnamed: 0'], inplace=True)
cust_info_df.drop(columns=['Unnamed: 0'], inplace=True)
cust_prod_df.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
cust_cases_df.head(), cust_info_df.head(), cust_prod_df.head(), prod_info_df.head()

(  case_id            date_time customer_id channel  reason
 0   CC101  2017-01-01 10:32:03       C2448   phone  signup
 1   CC102  2017-01-01 11:35:47       C2449   phone  signup
 2   CC103  2017-01-01 11:37:09       C2450   phone  signup
 3   CC104  2017-01-01 13:28:14       C2451   phone  signup
 4   CC105  2017-01-01 13:52:22       C2452   phone  signup,
   customer_id  age  gender
 0       C2448   76  female
 1       C2449   61    male
 2       C2450   58  female
 3       C2451   62  female
 4       C2452   71    male,
   customer_id product     signup_date_time     cancel_date_time
 0       C2448   prd_1  2017-01-01 10:35:09                  NaN
 1       C2449   prd_1  2017-01-01 11:39:29  2021-09-05 10:00:02
 2       C2450   prd_1  2017-01-01 11:42:00  2019-01-13 16:24:55
 3       C2451   prd_2  2017-01-01 13:32:08                  NaN
 4       C2452   prd_1  2017-01-01 13:57:30  2021-06-28 18:06:01,
   product_id                  name  price  billing_cycle
 0      prd_1   annua

In [7]:
# Merge customer_cases with customer_product on customer_id
merged_df = pd.merge(cust_cases_df, cust_prod_df, on='customer_id', how='left')

In [8]:
# Merge the result with customer_info on customer_id
merged_df = pd.merge(merged_df, cust_info_df, on='customer_id', how='left')

In [9]:
# Merge the result with product_info on product
churn_df = pd.merge(merged_df, prod_info_df, left_on='product', right_on='product_id', how='left')

In [10]:
churn_df.head()

,case_id,date_time,customer_id,channel,reason,product,signup_date_time,cancel_date_time,age,gender,product_id,name,price,billing_cycle
0,CC101,2017-01-01 10:32:03,C2448,phone,signup,prd_1,2017-01-01 10:35:09,NaN,76,female,prd_1,annual_subscription,1200,12
1,CC102,2017-01-01 11:35:47,C2449,phone,signup,prd_1,2017-01-01 11:39:29,2021-09-05 10:00:02,61,male,prd_1,annual_subscription,1200,12
2,CC103,2017-01-01 11:37:09,C2450,phone,signup,prd_1,2017-01-01 11:42:00,2019-01-13 16:24:55,58,female,prd_1,annual_subscription,1200,12
3,CC104,2017-01-01 13:28:14,C2451,phone,signup,prd_2,2017-01-01 13:32:08,NaN,62,female,prd_2,monthly_subscription,125,1
4,CC105,2017-01-01 13:52:22,C2452,phone,signup,prd_1,2017-01-01 13:57:30,2021-06-28 18:06:01,71,male,prd_1,annual_subscription,1200,12


In [11]:
# rename the column name to product_name
churn_df.rename(columns={'name': 'product_name'}, inplace=True)

In [12]:
# Ensure both columns are in datetime format
churn_df['signup_date_time'] = pd.to_datetime(churn_df['signup_date_time'])
churn_df['cancel_date_time'] = pd.to_datetime(churn_df['cancel_date_time'])

In [13]:
# Calculate days_since_signup
reference_time = pd.to_datetime(datetime.now())
churn_df['days_since_signup'] = (churn_df['cancel_date_time'].fillna(reference_time) - churn_df['signup_date_time']).dt.total_seconds() / 86400  # in days

In [14]:
# Create a outcome column using cancel_date_time
churn_df['churned'] = churn_df['cancel_date_time'].notnull()

In [15]:
# remove the unwanted columns
churn_df = churn_df.drop(columns=['case_id', 'customer_id', 'product', 'product_id', 'cancel_date_time', 'signup_date_time', 'date_time'])

In [16]:
churn_df.head()

,channel,reason,age,gender,product_name,price,billing_cycle,days_since_signup,churned
0,phone,signup,76,female,annual_subscription,1200,12,3055.158830,False
1,phone,signup,61,male,annual_subscription,1200,12,1707.930938,True
2,phone,signup,58,female,annual_subscription,1200,12,742.196470,True
3,phone,signup,62,female,monthly_subscription,125,1,3055.035925,False
4,phone,signup,71,male,annual_subscription,1200,12,1639.172581,True


## Now we created the final DataFrame for our model by using `customer_cases.csv`, `customer_info.csv`, `customer_product.csv` and `product_info.csv`
### 1. Let's perform data inspection

In [17]:
# check the number of rows and columns in a dataframe
churn_df.shape

(330512, 9)

In [18]:
# gather basic information from dataframe
churn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330512 entries, 0 to 330511
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   channel            330512 non-null  object 
 1   reason             330512 non-null  object 
 2   age                330512 non-null  int64  
 3   gender             330512 non-null  object 
 4   product_name       330512 non-null  object 
 5   price              330512 non-null  int64  
 6   billing_cycle      330512 non-null  int64  
 7   days_since_signup  330512 non-null  float64
 8   churned            330512 non-null  bool   
dtypes: bool(1), float64(1), int64(3), object(4)
memory usage: 20.5+ MB


### 2. Check the null count in each column

In [19]:
churn_df.isnull().sum()

channel              0
reason               0
age                  0
gender               0
product_name         0
price                0
billing_cycle        0
days_since_signup    0
churned              0
dtype: int64

### 3. Check the duplicate records in a dataframe if exist just drop the duplicate records

In [20]:
churn_df.duplicated().sum()

np.int64(16594)

In [21]:
churn_df.drop_duplicates(inplace=True)

In [22]:
# store the column names in columns variable
columns = churn_df.columns
print(f'churn DataFrame columns: {columns}')

churn DataFrame columns: Index(['channel', 'reason', 'age', 'gender', 'product_name', 'price',
       'billing_cycle', 'days_since_signup', 'churned'],
      dtype='object')


In [23]:
# Threshold for "low uniqueness" (e.g., less than 10% unique values)
threshold = 0.0001

# Total number of rows
n_rows = len(churn_df)

# Create list of categorical columns with low uniqueness
cat_col = [
    col for col in churn_df.columns
    if churn_df[col].nunique() / n_rows < threshold
]

print("Categorical columns with low uniqueness:", cat_col)


Categorical columns with low uniqueness: ['channel', 'reason', 'gender', 'product_name', 'price', 'billing_cycle', 'churned']


In [24]:
# check the unique values each column
for col in columns:
    print(f'{col} unique values: {churn_df[col].unique()}')

channel unique values: ['phone' 'email']
reason unique values: ['signup' 'support']
age unique values: [76 61 58 62 71 56 53 70 68 64 54 42 34 51 55 40 52 46 66 65 30 37 41 73
 50 75 45 39 59 69 49 78 72 63 43 57 24 48 47 60 33 27 44 74 35 28 67 77
 38 36 32 25 26 31 23 29 22]
gender unique values: ['female' 'male']
product_name unique values: ['annual_subscription' 'monthly_subscription']
price unique values: [1200  125]
billing_cycle unique values: [12  1]
days_since_signup unique values: [3055.15883024 1707.9309375   742.19646991 ... 1297.00792746 1297.0449645
 1344.02464042]
churned unique values: [False  True]


### 4. Label the categorical columns

In [25]:
category_mappings = {}  # To store column-wise mappings
encoder = LabelEncoder()

for column in cat_col:
  encoder.fit(churn_df[column].unique())
  churn_df[column] = encoder.transform(churn_df[column])

  # Store mapping: category -> encoded value
  mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
  category_mappings[column] = mapping

In [26]:
# DataFrame after performing label Encoding
churn_df.head()

,channel,reason,age,gender,product_name,price,billing_cycle,days_since_signup,churned
0,1,0,76,0,0,1,1,3055.158830,0
1,1,0,61,1,0,1,1,1707.930938,1
2,1,0,58,0,0,1,1,742.196470,1
3,1,0,62,0,1,0,0,3055.035925,0
4,1,0,71,1,0,1,1,1639.172581,1


### 5. Split the training and testing data

In [27]:
# Split the input as features and outcome
feature, outcome = churn_df.drop(columns=['churned']), churn_df['churned']

feature_train, feature_test, outcome_train, outcome_test = train_test_split(feature, outcome, test_size=0.2, random_state=42)

### 6. Standerdize the feature

In [28]:
scaler = StandardScaler()

feature_train = scaler.fit_transform(feature_train)
feature_test = scaler.fit_transform(feature_test)

### 7. Create the model

In [29]:
model = LogisticRegression()

### 8. Train the model

In [30]:
model.fit(feature_train, outcome_train)

LogisticRegression()

### 9. Evaluate the model
#### 9.1 Let's calculate the Training Accuracy

In [31]:
# Predict the values using training data for calculating training accuracy
feature_train_pred = model.predict(feature_train)
feature_train_proba = model.predict_proba(feature_train)[:, 1]  # Probabilities for ROC-AUC

In [32]:
# create a evaluation dictionary
eve_metrics = ['Precision', 'Recall', 'F1 Score', 'ROC-AUC Score']

# Evaluation Metrics
precision = precision_score(outcome_train, feature_train_pred)
recall = recall_score(outcome_train, feature_train_pred)
f1 = f1_score(outcome_train, feature_train_pred)
roc_auc = roc_auc_score(outcome_train, feature_train_proba)
conf_matrix = confusion_matrix(outcome_train, feature_train_pred)

In [33]:
# Create dictionary using zip
eve_metrics_dict = {}
eve_metrics_dict['Train Metrics'] = dict(zip(eve_metrics, [precision, recall, f1, roc_auc]))
pd.DataFrame(eve_metrics_dict)

,Train Metrics
Precision,0.996219
Recall,0.946800
F1 Score,0.970881
ROC-AUC Score,0.993496


In [34]:
pd.DataFrame(conf_matrix)

,0,1
0,192767,209
1,3094,55064


#### 9.2 Let's check the testing accuracy

In [35]:
# Predict the values using testing data for calculating testing accuracy
feature_test_pred = model.predict(feature_test)
feature_test_proba = model.predict_proba(feature_test)[:, 1]  # Probabilities for ROC-AUC

In [36]:
# Evaluation Metrics
precision = precision_score(outcome_test, feature_test_pred)
recall = recall_score(outcome_test, feature_test_pred)
f1 = f1_score(outcome_test, feature_test_pred)
roc_auc = roc_auc_score(outcome_test, feature_test_proba)
conf_matrix = confusion_matrix(outcome_test, feature_test_pred)

In [37]:
# Create dictionary using zip
eve_metrics_dict['Test Metrics'] = dict(zip(eve_metrics, [precision, recall, f1, roc_auc]))
eve_metrics_df = pd.DataFrame(eve_metrics_dict)
eve_metrics_df

,Train Metrics,Test Metrics
Precision,0.996219,0.995504
Recall,0.946800,0.948587
F1 Score,0.970881,0.971479
ROC-AUC Score,0.993496,0.994118


In [38]:
pd.DataFrame(conf_matrix)

,0,1
0,48251,62
1,744,13727


In [54]:
# Columns to expect in this exact order
input_columns = [
    'channel', 'reason', 'age', 'gender',
    'product_name', 'price', 'billing_cycle', 'days_since_signup'
]

# Assume category_mappings already exists as you set earlier
# {'channel': {'online': 0, 'offline': 1}, ... }

categorical_cols = list(category_mappings.keys())
numerical_cols = [col for col in input_columns if col not in categorical_cols]

In [55]:
def predict_churn_with_mapping(*args):
    input_dict = {}
    i = 0
    # Categorical features
    for col in categorical_cols:
        val = args[i]
        if val not in category_mappings[col]:
            return f"Invalid input for {col}"
        input_dict[col] = category_mappings[col][val]
        i += 1
    # Numerical features
    for col in numerical_cols:
        input_dict[col] = args[i]
        i += 1

    # Construct input DataFrame in the right order
    input_df = pd.DataFrame([[input_dict[col] for col in input_columns]], columns=input_columns)

    # Scale and predict
    input_scaled = scaler.transform(input_df)
    pred = model.predict(input_scaled)[0]
    prob = model.predict_proba(input_scaled)[0][1]
    return f"Churn Prediction: {'Yes' if pred else 'No'} (Probability: {prob:.2f})"

In [56]:
# Gradio Inputs
gr_inputs = []

for col in categorical_cols:
    gr_inputs.append(gr.Dropdown(choices=list(category_mappings[col].keys()), label=col))

for col in numerical_cols:
    gr_inputs.append(gr.Number(label=col))

In [57]:
# Launch the Gradio app
gr.Interface(
    fn=predict_churn_with_mapping,
    inputs=gr_inputs,
    outputs="text",
    title="Customer Churn Predictor",
    description="Provide customer attributes to get churn prediction"
).launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
